# Relatório de Avaliação de Modelos para Armazenamento de Áudio em Bases de Dados Vetoriais (Milvus)

### Objetivo do Projeto

Este relatório tem como objetivo apresentar uma avaliação comparativa (benchmark) de diferentes modelos de processamento de áudio para armazenamento em bases de dados vetoriais, utilizando a tecnologia Milvus. Ao longo do notebook interativo, são analisados os desempenhos dos distintos modelos e embeddings, com o intuito de identificar qual abordagem oferece melhores resultados para o nosso caso de uso. No final do processo, será possível selecionar o modelo ou embedding mais adequado com base em critérios de eficiência, fidelidade e compatibilidade com a estrutura vetorial da base de dados.



## Introdução

### Introdução

O presente trabalho tem como objetivo principal a realização de um benchmark de diferentes modelos de extração de embeddings a partir de ficheiros de áudio, com vista à sua posterior indexação e recuperação eficiente em bases de dados vetoriais, utilizando a plataforma Milvus.

Para isso, foi construído um pipeline modular e interativo, implementado em ambiente Jupyter Notebook, que permite:

- A leitura e pré-processamento de ficheiros de áudio;
- A extração de embeddings através de diferentes modelos de deep learning.
- A visualização e análise exploratória dos embeddings gerados utilizando técnicas como **t-SNE**;
- O armazenamento e recuperação vetorial através da base de dados **Milvus**, com suporte à consulta por similaridade;
- A apresentação dos resultados através de um **dashboard interativo**, facilitando a comparação visual e quantitativa entre diferentes abordagens.

O objetivo final é determinar qual dos modelos testados oferece a melhor performance em termos de coerência semântica dos embeddings, velocidade de processamento, compressão e desempenho na recuperação de ficheiros semelhantes. Esta análise permitirá selecionar a arquitetura mais adequada às necessidades específicas do projeto, garantindo eficiência na representação vetorial de áudio.



## Falar o que fizemos para fazer o benchmark

## Metodologia do Benchmark

Para realizar o benchmark entre diferentes modelos de extração de embeddings de áudio, foi seguido um processo estruturado composto pelas seguintes etapas:

1. **Pré-processamento dos Ficheiros de Áudio**  
   Os ficheiros de áudio foram carregados e convertidos para um formato padronizado  para garantir compatibilidade com os modelos utilizados.

2. **Extração de Embeddings com Múltiplos Modelos**  
   Foram utilizados diferentes modelos baseados na arquitetura **Wav2Vec2** (da Hugging Face) para gerar embeddings representativos do conteúdo dos áudios. Cada modelo processou os mesmos ficheiros, permitindo uma comparação justa.

3. **Visualização com Redução de Dimensionalidade**  
   Para análise qualitativa dos embeddings, aplicámos o algoritmo **t-SNE** de redução de dimensionalidade. Com isto, foi possível projetar os embeddings para um espaço bidimensional, facilitando a inspeção visual de agrupamentos e dispersões.

4. **Indexação Vetorial com Milvus**  
   Os embeddings gerados por cada modelo foram armazenados numa instância da base de dados vetorial **Milvus**, que permite consultas por similaridade (busca de áudios semelhantes).

5. **Dashboard Interativo de Avaliação**  
   Foi desenvolvido um dashboard em Jupyter Notebook para comparar os diferentes modelos, com base em métricas de desempenho e visualizações interativas dos clusters gerados. O dashboard permite ao utilizador filtrar modelos, inspecionar os embeddings, ouvir os áudios associados e observar o comportamento da busca vetorial.

Este fluxo de trabalho permitiu uma comparação clara, interativa e reprodutível entre os diferentes modelos de embeddings, facilitando a escolha da abordagem mais eficaz para tarefas de recuperação de áudio baseada em conteúdo.


## Fases do Projeto

O desenvolvimento deste projeto foi dividido em várias fases, permitindo uma abordagem organizada e iterativa no processo de benchmark de modelos de áudio. As etapas principais foram as seguintes:

1. **Definição do Objetivo e Seleção de Ferramentas**  
   Começou-se por definir o objetivo do projeto: comparar diferentes modelos de extração de embeddings de áudio para armazenamento e consulta em bases de dados vetoriais. Foram selecionadas ferramentas como Python, Jupyter Notebook, `librosa`, `transformers`, `Milvus`, `scikit-learn`, `plotly`, entre outras.

2. **Aquisição e Pré-processamento de Áudio**  
   Foram utilizados ficheiros de áudio de diferentes categorias. Os ficheiros foram normalizados para formato mono com taxa de amostragem de 16 kHz, assegurando a uniformidade dos dados para entrada nos modelos.

3. **Implementação de Vários Modelos de Embeddings de Áudio**  
   Foram testados diversos modelos de extração de embeddings, incluindo arquiteturas baseadas em **Wav2Vec2**, **Hubert**, **Whisper** e outras alternativas disponíveis na biblioteca `transformers` e fontes externas. Cada modelo foi utilizado para gerar representações vetoriais a partir dos mesmos áudios.

4. **Indexação em Base de Dados Vetorial (Milvus)**  
   Os embeddings obtidos foram armazenados e indexados numa instância do **Milvus**, permitindo a realização de buscas por similaridade entre vetores de áudio.

5. **Visualização com Redução de Dimensionalidade**  
   Foi aplicada a técnica **t-SNE** para reduzir a dimensionalidade dos embeddings, permitindo visualizar a distribuição e agrupamento dos áudios num espaço bidimensional.

6. **Desenvolvimento de um Dashboard Interativo**  
   Criou-se um dashboard interativo em Jupyter com `plotly`, `ipywidgets` e outros recursos, permitindo a comparação visual dos modelos, escuta dos áudios originais e exploração dos resultados de forma dinâmica.

7. **Avaliação e Conclusão**  
   Por fim, foram analisadas as características de cada modelo — como separabilidade dos embeddings, fidelidade, robustez e comportamento na busca por similaridade — com o objetivo de identificar a solução mais adequada ao contexto do projeto.



## Porque o Milvus?

A escolha do **Milvus** como base de dados vetorial para este projeto foi motivada por várias razões técnicas e práticas que o tornam particularmente adequado para aplicações que envolvem pesquisa por similaridade em grandes volumes de dados não estruturados, como áudio.

As principais razões para a sua escolha foram:

- **Desempenho e Escalabilidade**  
  O Milvus é otimizado para buscas vetoriais de alta performance, suportando milhões (ou até bilhões) de vetores com tempos de resposta muito baixos, graças à sua integração com mecanismos de indexação como IVF, HNSW e PQ.

- **Compatibilidade com Workflows de Machine Learning**  
  Possui uma API moderna e integração nativa com ferramentas amplamente usadas em ciência de dados e IA, como `PyMilvus`, `FAISS` e `Docker`, o que facilita a sua incorporação em pipelines de machine learning.

- **Suporte a Vários Tipos de Dados**  
  Embora este projeto se foque em áudio, o Milvus permite o armazenamento vetorial de outros tipos de dados não estruturados, como imagens, vídeos e texto, o que abre possibilidades futuras de expansão multimodal.

- **Interface Flexível e Documentação Completa**  
  A API do Milvus é bem documentada, com suporte a gRPC, REST e bibliotecas Python, o que simplifica o desenvolvimento e a integração em notebooks interativos como o utilizado neste projeto.

- **Open Source e Ativamente Mantido**  
  O projeto Milvus é open-source e possui uma comunidade ativa, garantindo constante evolução da plataforma e liberdade tecnológica para adaptar soluções a diferentes casos de uso.

Em resumo, o Milvus foi escolhido por ser uma solução robusta, escalável e eficiente para realizar indexação e busca por similaridade em vetores de embeddings, representando uma escolha sólida para este tipo de benchmark com dados de áudio.


### Embeddings Escolhidos

Para este benchmark foram selecionados diferentes modelos de extração de embeddings de áudio, com arquiteturas e características distintas. A diversidade dos modelos permite avaliar diferentes formas de representação vetorial do áudio, desde características perceptivas até representações semânticas mais abstratas.

Os modelos utilizados foram os seguintes:

- **Wav2Vec2** (768 dimensões)  
  Um modelo auto-supervisionado da Meta AI, treinado para representação de fala. É particularmente eficaz em tarefas relacionadas com transcrição e compreensão semântica do conteúdo vocal.

- **VGGish** (128 dimensões)  
  Um modelo inspirado na arquitetura VGG da área de visão computacional, adaptado ao domínio áudio. Foi treinado com dados do YouTube-8M, sendo adequado para tarefas de classificação sonora geral.

- **OpenL3** (512 dimensões)  
  Baseado em embeddings aprendidos de forma multimodal (áudio + vídeo), o OpenL3 captura representações semânticas e perceptivas, mostrando-se eficaz em várias tarefas, como identificação de sons ambientais e música.

- **YAMNet** (1024 dimensões)  
  Modelo baseado em MobileNetV1, treinado no AudioSet da Google. Produz embeddings de alta dimensionalidade que capturam informações de classificação sonora com elevado detalhe.

- **CLAP** (512 dimensões)  
  Contrastive Language-Audio Pretraining — um modelo semelhante ao CLIP, mas treinado para alinhar texto e áudio. Permite gerar embeddings compatíveis com descrições textuais, útil para tarefas multimodais e de recuperação baseada em linguagem.

- **AST (Audio Spectrogram Transformer)** (768 dimensões)  
  Modelo baseado em transformadores aplicado diretamente sobre espectrogramas de áudio. Alcança desempenho de topo em benchmarks como ESC-50 e AudioSet, com uma arquitetura inspirada no Vision Transformer (ViT).

Cada modelo foi integrado no pipeline de forma modular, permitindo a extração dos embeddings diretamente a partir dos ficheiros de áudio. A escolha destes modelos visa abranger diferentes estratégias de representação vetorial, de forma a identificar qual delas é mais eficiente e semanticamente útil para armazenamento e recuperação vetorial com Milvus.


### Script de Benchmark

O script `audio_benchmark.py` foi desenvolvido como núcleo do processo de avaliação dos modelos de embeddings de áudio. Ele implementa um pipeline completo e automatizado para extração de embeddings, medição de desempenho, indexação vetorial e análise dos resultados. A arquitetura modular e robusta do script garante flexibilidade, reprodutibilidade e resiliência em diferentes ambientes de execução.

#### 1. Importações e Carregamento Inteligente de Modelos

O carregamento dos modelos ocorre uma única vez no início do script, minimizando o uso de recursos e evitando redundância. Foram usados modelos diversos, com diferentes origens (Hugging Face e TensorFlow Hub), e foram incluídos mecanismos de fallback (como modelos "dummy") para garantir a continuidade do processamento mesmo em caso de falhas.

#### 2. Extração de Embeddings

Cada modelo conta com a sua própria função de extração, adaptada à sua interface específica e requisitos (ex: taxa de amostragem). As funções são projetadas para retornar embeddings de dimensão fixa, padronizados para indexação.

#### 3. Benchmarking e Medição de Recursos

A função `benchmark_embeddings(...)` executa todo o ciclo de benchmarking:
- Carrega e normaliza os ficheiros de áudio;
- Extrai embeddings com vários modelos;
- Mede tempo de extração, inserção, uso de CPU, RAM, GPU e memória máxima (com `tracemalloc`);
- Insere os dados na base de dados vetorial **Milvus**, com metadados como duração, autores, sample rate, etc.

Todos os resultados são armazenados numa estrutura de `DataFrame`, depois exportada como `.csv`.

#### 4. Análise e Visualização

A função `analisar_resultados(...)` gera gráficos estatísticos e visuais de comparação entre os modelos, incluindo:
- Tempos médios de extração e inserção;
- Uso de memória;
- Correlações entre duração/tamanho dos ficheiros e tempo de processamento;
- Evolução de recursos (CPU, RAM, GPU);
- Boxplots e scatterplots tridimensionais.

Os gráficos são automaticamente guardados numa pasta específica para o benchmark, garantindo documentação completa e organizada.

#### 5. Benchmark de Pesquisa Vetorial

A função `testar_pesquisas(...)` permite avaliar a qualidade prática dos embeddings, realizando buscas por similaridade diretamente no Milvus. São recuperados os resultados com maior score de semelhança e é gerado um gráfico comparativo por modelo.

#### 6. Automação e Organização

Além da lógica principal, o script oferece uma série de funcionalidades auxiliares que reforçam a qualidade técnica do projeto:

- **Criação automática de diretórios** para guardar os resultados de cada benchmark;
- **Geração de ficheiros `.csv`** com os dados brutos do benchmarking;
- **Logging estruturado** (`benchmark.log`) com mensagens detalhadas sobre o estado de execução, falhas, tempos e recursos utilizados.

#### 7. Destaques Técnicos do Script

- **Resiliência**: Mesmo que um modelo falhe ao carregar, o script continua com alternativas seguras (`dummy models`);
- **Reprodutibilidade**: Todas as métricas e resultados são guardados em diretórios organizados, permitindo comparar diferentes execuções;
- **Extensibilidade**: É simples adicionar novos modelos ou alterar parâmetros sem comprometer a estrutura;
- **Eficiência**: Os modelos são carregados apenas uma vez; os recursos são monitorizados de forma leve, e o uso de `tqdm` melhora a experiência de execução;
- **Completo**: O script cobre todas as fases do benchmark — da extração à análise final, passando pela indexação e pesquisa vetorial.

#### 8. Controlo do Ambiente com Docker

Para garantir que todas as execuções fossem realizadas sob **condições de hardware e software idênticas**, foi criado um **container Docker dedicado**. Esta abordagem garante:

- Isolamento total do ambiente de execução;
- Reprodutibilidade absoluta dos benchmarks;
- Neutralização de variações externas (ex: dependências, drivers, bibliotecas);
- Facilidade de deployment em diferentes máquinas ou servidores.

O container inclui todas as dependências necessárias (Python, PyTorch, TensorFlow, Milvus client, etc.) e pode ser executado com um simples comando, tornando o sistema portável e escalável.

---

Este script, juntamente com o ambiente Dockerizado, representa uma infraestrutura profissional para benchmarking de embeddings de áudio, e pode ser facilmente reutilizado ou adaptado para tarefas futuras envolvendo vídeo, texto, ou outras modalidades multimodais.


## Análise dos Resultados

Nesta secção, serão apresentados e analisados os resultados obtidos a partir do benchmark realizado com os diferentes modelos de embeddings de áudio. O objetivo é comparar o desempenho de cada modelo com base em métricas quantitativas e qualitativas, tais como:

- **Tempo médio de extração dos embeddings**;
- **Tempo médio de inserção na base de dados vetorial (Milvus)**;
- **Pico de utilização de memória RAM durante a extração**;
- **Correlação entre duração ou tamanho dos ficheiros e o tempo de processamento**;
- **Eficiência da pesquisa vetorial (score dos resultados de similaridade)**;
- **Evolução do uso de CPU, RAM e GPU ao longo do processamento**.

Serão também apresentados gráficos e visualizações interativas (scatterplots, boxplots, barras, 3D), permitindo observar tendências, outliers e padrões de comportamento entre os diferentes modelos.

O objetivo desta análise é identificar quais modelos apresentam melhor equilíbrio entre desempenho computacional e qualidade dos embeddings, ajudando a fundamentar a escolha do modelo mais adequado ao nosso caso de uso.



## Instalação de Dependências e Configuração do Ambiente

Para executar o notebook ou o script de benchmark, é necessário instalar as dependências listadas no ficheiro `requirements_jupyter.txt`. Para isso, basta utilizar o seguinte comando:

```bash
pip install -r requirements_jupyter.txt


In [1]:
!pip install -r requirements_jupyter.txt

### 1. Importações e Preparação do Ambiente

Nesta primeira etapa, foram realizadas as importações das bibliotecas essenciais para o desenvolvimento do projeto. As bibliotecas englobam ferramentas para análise e visualização de dados (`pandas`, `numpy`, `matplotlib`, `seaborn`, `plotly`), redução de dimensionalidade (`sklearn.manifold.TSNE`), manipulação de arquivos e objetos (`ast`, `io`, `soundfile`, `IPython.display`, `ipywidgets`), bem como processamento de áudio e machine learning (`librosa`, `torch`, `transformers`, `MilvusClient`). Esta preparação permite a manipulação eficiente de dados, visualizações interativas e a aplicação de modelos de deep learning, nomeadamente com a arquitetura Wav2Vec2 para tarefas de processamento de áudio.


In [2]:
# 📦 Importações e leitura dos dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.manifold import TSNE
import ast
from pymilvus import MilvusClient
import librosa
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import io
from IPython.display import display
import ipywidgets as widgets




### 2º Carregamento dos Dados

Após a execução do script de benchmark, os resultados são guardados em ficheiros `.csv`, organizados em diretórios específicos dentro da pasta `benchmarks/`. Cada pasta representa uma execução independente, identificada pelo número de ficheiros processados (ex: `benchmark_30`, `benchmark_250`, etc.).

Nesta fase, o notebook carrega os dados do ficheiro `resultados_benchmark.csv`, presente no diretório da execução. Os dados contêm métricas detalhadas de performance para cada modelo de embeddings, além dos próprios vetores (`embedding`), que são convertidos de string para listas numéricas com `ast.literal_eval`.

Com o crescimento do projeto, será necessário comparar múltiplos benchmarks realizados em diferentes contextos (ex: com 30, 100, 250 ficheiros, ou com ficheiros de diferentes tipos de áudio).  
A estrutura modular adotada permite facilmente carregar múltiplos datasets, bastando iterar sobre diferentes diretórios do tipo `benchmarks/benchmark_X/`.

No futuro, será possível:

- **Unificar e comparar múltiplos benchmarks num único DataFrame**;
- **Aplicar filtros dinâmicos por modelo, dataset ou número de ficheiros**;
- **Visualizar a evolução do desempenho com o aumento da escala de dados**;
- **Integrar esses benchmarks numa interface (ex: dashboard Streamlit)** que permita ao utilizador selecionar o conjunto de resultados a analisar.

Esta abordagem garante **escalabilidade horizontal**, permitindo expandir o sistema para novos testes e conjuntos de dados sem alterações estruturais ao código principal.


In [3]:
# 📂 Leitura e limpeza dos dados
try:
    df = pd.read_csv('benchmarks/benchmark_250/resultados_benchmark.csv')
    df['embedding'] = df['embedding'].apply(ast.literal_eval)
    
    # Success message with dataset information
    print(f"✅ Dados carregados com sucesso!")
    print(f"   - {len(df)} registros de embeddings carregados")
    print(f"   - Modelos disponíveis: {', '.join(df['modelo'].unique())}")
    
except FileNotFoundError as e:
    print(f"❌ Erro ao carregar os arquivos de benchmark: {e}")
    print("   Verifique se os arquivos estão no diretório correto: benchmarks/benchmark_30/")
except Exception as e:
    print(f"❌ Erro ao processar os dados: {e}")
    print("   Verifique o formato dos arquivos CSV e tente novamente.")

✅ Dados carregados com sucesso!
   - 1500 registros de embeddings carregados
   - Modelos disponíveis: wav2vec2, vggish, openl3, yamnet, clap, ast


### Conexão com Milvus e Seleção de Coleções

Antes de realizar qualquer operação de pesquisa vetorial, é necessário estabelecer uma conexão com a instância do **Milvus**, que deve estar em execução localmente (por padrão, em `http://localhost:19530`).

O código tenta criar um cliente `MilvusClient` apontando para o URI local. Em seguida, verifica se existem coleções já criadas no servidor, listando-as e apresentando informações como:

- Número total de coleções disponíveis;
- Nomes das coleções encontradas (ex: `audio_wav2vec2_30`, `audio_openl3_250`, etc.).

Caso a conexão falhe, são apresentadas mensagens de erro específicas, incluindo sugestões para resolver o problema — por exemplo, iniciar o Milvus com `docker-compose up -d`. O tratamento de exceções garante que o script não falha abruptamente, e a variável `client` é definida como `None` caso a conexão não seja estabelecida com sucesso.

Esta etapa é fundamental para confirmar que o ambiente está corretamente preparado para realizar pesquisas por similaridade, recuperar vetores ou executar benchmarks adicionais.


In [4]:
# 🔄 Conexão com Milvus e seleção da coleção
try:
    # Tentar conectar ao Milvus
    client = MilvusClient(uri='http://localhost:19530')
    
    # Obter coleções disponíveis
    colecoes_disponiveis = client.list_collections()
    
    if colecoes_disponiveis:
        print(f"✅ Conexão com Milvus estabelecida com sucesso!")
        print(f"   - {len(colecoes_disponiveis)} coleções disponíveis")
        print(f"   - Coleções: {', '.join(colecoes_disponiveis)}")
        
    else:
        print("⚠️ Conectado ao Milvus, mas nenhuma coleção encontrada.")
        print("   Verifique se os dados foram carregados corretamente.")
        
except ConnectionError as e:
    print(f"❌ Erro de conexão com o Milvus: {e}")
    print("   Verifique se o servidor Milvus está em execução em http://localhost:19530")
    print("   Para iniciar o Milvus localmente com Docker, execute:")
    print("   docker-compose up -d")
    client = None
    
except Exception as e:
    print(f"❌ Erro ao conectar ao Milvus: {e}")
    print("   Verifique sua configuração e tente novamente.")
    client = None

2025-05-25 22:21:18,921 [ERROR][_create_connection]: Failed to create new connection using: 8f974fdfeff541b186b233eb1a269103 (milvus_client.py:923)


❌ Erro ao conectar ao Milvus: <MilvusException: (code=2, message=Fail connecting to server on localhost:19530, illegal connection params or server unavailable)>
   Verifique sua configuração e tente novamente.


### 4º Análise dos Resultados

Nesta fase do projeto, será realizada uma análise detalhada dos resultados obtidos durante o benchmark dos diferentes modelos de embeddings de áudio. O objetivo é avaliar e comparar o desempenho dos modelos com base em métricas quantitativas, como tempo, consumo de memória e eficácia na pesquisa vetorial.

A análise será conduzida utilizando **visualizações gráficas** que permitem identificar padrões, comparar desempenhos e apoiar a tomada de decisão sobre o modelo mais adequado. Os principais gráficos gerados são:

- 📊 **Gráficos de Barras (Barplots)**  
  Utilizados para visualizar o tempo médio de extração e inserção de embeddings por modelo, assim como o pico de memória RAM consumido durante a extração.

- 📦 **Boxplots**  
  Representam a distribuição dos tempos de extração por modelo, permitindo observar a variabilidade e identificar outliers no processamento.

- 🧠 **Scatterplots**  
  Avaliam a relação entre:
  - Duração do ficheiro de áudio e o tempo de extração;
  - Tamanho do ficheiro e tempo de extração;
  Estes gráficos ajudam a perceber como diferentes características dos dados influenciam a performance dos modelos.

- 📈 **Gráficos Temporais de Uso de Recursos**  
  Mostram a evolução do uso de CPU, RAM e GPU ao longo do processamento dos ficheiros. Isto permite identificar picos de utilização e comparar a eficiência dos modelos em tempo real.

- 📉 **Gráficos de Score da Pesquisa Vetorial**  
  Após a indexação no Milvus, é feita uma pesquisa por similaridade e analisado o *score* do primeiro resultado obtido por modelo. Um gráfico de barras compara diretamente esses scores, indicando quais embeddings produzem buscas mais precisas.

- 🧩 **Scatterplot 3D (opcional)**  
  Em casos com variabilidade suficiente, é possível gerar uma visualização tridimensional combinando tempo de extração, memória usada e tamanho do ficheiro, permitindo uma análise multivariada da eficiência dos modelos.

Todas estas visualizações serão geradas automaticamente e guardadas em ficheiros de imagem, organizadas numa pasta específica (`benchmarks/benchmark_X/`). A análise será acompanhada por resumos estatísticos (média, desvio padrão) e insights qualitativos que ajudarão a fundamentar a escolha do melhor modelo.



### Benchmarks Interativos

Foi desenvolvido um **dashboard interativo com recurso ao Streamlit**, que permite visualizar e explorar os resultados dos benchmarks realizados com diferentes volumes de dados (5, 25, 50, 100 e 250 ficheiros de áudio). Esta interface facilita a análise comparativa entre modelos, permitindo filtrar por modelo, métrica ou tamanho da base de dados de forma dinâmica e intuitiva.

No entanto, para efeitos deste relatório, **optamos por focar a análise nos resultados obtidos com a base de dados contendo 250 ficheiros**, uma vez que este cenário oferece uma visão mais robusta e representativa do desempenho real dos modelos. Esta abordagem permite avaliar o comportamento dos modelos em contexto de maior escala, destacando diferenças de performance que não seriam tão evidentes em conjuntos de menor dimensão.


Para executar o dashboard interativo, basta instalar o Streamlit com o seguinte comando:

```bash
pip install streamlit

Caso pretenda executar o dashboard, correr os comandos abaixo

In [ ]:
#!pip install streamlit

In [ ]:
#Caso pretenda correr o dashboard interativo
#streamlit run dashboard.py


### Análise dos Gráficos Interativos por Métrica

A interface interativa apresenta os resultados em diferentes **abas**, cada uma representando uma métrica específica relacionada com o desempenho dos modelos de extração de embeddings de áudio. Abaixo, detalhamos o objetivo de cada gráfico e as principais observações que podem ser extraídas.

---

#### 1️⃣ **Tempo de Extração por Modelo** (`fig1`)

**Tipo:** Gráfico de barras  
**Métrica:** `tempo_extracao` (segundos)

Este gráfico apresenta o **tempo médio necessário por cada modelo para extrair embeddings de áudio**.  
- ✅ Modelos com barras mais baixas são mais rápidos e eficientes.


---

#### 2️⃣ **Tempo de Inserção no Milvus por Modelo** (`fig2`)

**Tipo:** Gráfico de barras  
**Métrica:** `tempo_insercao` (segundos)

Este gráfico mostra o **tempo médio necessário para inserir os embeddings na base de dados Milvus**.  
- Diferenças nesta métrica podem refletir a **dimensão do vetor** gerado por cada modelo (ex: YAMNet gera vetores com 1024 dimensões).
- Modelos com **dimensões maiores** tendem a demorar mais.

---

#### 3️⃣ **Boxplot do Tempo de Extração** (`fig3`)

**Tipo:** Boxplot  
**Métrica:** `tempo_extracao` (distribuição)

Este gráfico permite observar a **variabilidade** dos tempos de extração por modelo:  
- ✅ Modelos com boxplots mais “estreitos” são mais **consistentes e previsíveis**.
- ⚠️ Modelos com muitos **outliers ou dispersão elevada** podem ter desempenho instável em diferentes tipos de áudio.

---

#### 4️⃣ **Boxplot do Tempo de Inserção** (`fig4`)

**Tipo:** Boxplot  
**Métrica:** `tempo_insercao` (distribuição)

Tal como o anterior, este gráfico permite verificar a **estabilidade** no tempo de inserção dos vetores na base de dados.  
- Pode indicar se **certos ficheiros causam atrasos** em modelos específicos.

---

#### 5️⃣ **Tamanho do Áudio vs Tempo de Extração** (`fig5`)

**Tipo:** Gráfico de dispersão (scatter plot)  
**Eixos:** `tamanho_audio_mb` vs `tempo_extracao`

Este gráfico mostra a correlação entre o **tamanho (em MB)** dos ficheiros de áudio e o **tempo que cada modelo leva para extrair o embedding**.  
- Ideal para verificar **escalabilidade linear**.
- ✅ Uma relação linear indica eficiência previsível.
- ⚠️ Curvas ou dispersões irregulares podem indicar problemas de desempenho com ficheiros maiores.

---

#### 6️⃣ **Tamanho do Áudio vs Tempo de Inserção** (`fig6`)

**Tipo:** Gráfico de dispersão  
**Eixos:** `tamanho_audio_mb` vs `tempo_insercao`

Análogo ao gráfico anterior, mas focado no tempo de inserção dos dados no Milvus.  
- Pode ajudar a perceber se a **inserção é afetada pela complexidade ou dimensão dos vetores** oriundos de ficheiros maiores.

---

### Conclusão Geral

- Modelos como **VGGish** tendem a ser mais rápidos e estáveis, mas podem sacrificar riqueza semântica.
- Modelos como **YAMNet**, **CLAP** e **AST** oferecem mais detalhe nos embeddings, mas com **maior custo computacional**.
- **Boxplots** ajudam a detetar **instabilidades**, e os gráficos de dispersão revelam **como os modelos escalam** com ficheiros maiores.
- A escolha ideal de modelo dependerá do **balanço entre velocidade e qualidade** desejada para a aplicação final (tempo real, precisão semântica, consumo de recursos, etc.).



In [6]:
# 📊 Gráficos interativos por métrica em abas
tab_contents = ['Extração', 'Inserção', 'Boxplot Extração', 'Boxplot Inserção','Tamanho vs Extração','Tamanho vs Inserção' ]
children = []

fig1 = px.bar(df, x='modelo', y='tempo_extracao', color='modelo', title='Tempo de Extração por Modelo')
fig2 = px.bar(df, x='modelo', y='tempo_insercao', color='modelo', title='Tempo de Inserção por Modelo')
fig3 = px.box(df, x='modelo', y='tempo_extracao', color='modelo', title='Boxplot Tempo de Extração')
fig4 = px.box(df, x='modelo', y='tempo_insercao', color='modelo', title='Boxplot Tempo de Inserção')
fig5 = px.scatter(df, x='tamanho_audio_mb', y='tempo_extracao', color='modelo', title='Tamanho vs Extração')
fig6 = px.scatter(df, x='tamanho_audio_mb', y='tempo_insercao', color='modelo', title='Tamanho vs Inserção')

#ADICIONAR MAIS ? 

for fig in [fig1, fig2, fig3, fig4,fig5, fig6]:
    # Create a new tab for each figure
    out = widgets.Output()
    with out:
        display(fig)
    children.append(out)

tab = widgets.Tab(children=children)
for i in range(len(tab_contents)):
    tab.set_title(i, tab_contents[i])
display(tab)

## 🧪 Comparação Final dos Modelos com Base nos Resultados dos Gráficos

Com base na análise visual dos gráficos interativos, avaliamos o desempenho relativo de cada modelo de embeddings de áudio nas principais métricas: tempo de extração, tempo de inserção, estabilidade e escalabilidade. Abaixo segue uma análise detalhada por métrica, culminando na eleição do melhor e do pior modelo do benchmark.

---

### 🏆 Melhor Modelo: **VGGish**

### ❌ Pior Modelo: **OpenL3**

---

### 📊 Análise Métrica a Métrica

---

#### 1️⃣ Tempo de Extração por Modelo

- **VGGish**, **Wav2Vec2**, **YAMNet**, **CLAP** e **AST** apresentaram tempos de extração baixos e consistentes.
- **OpenL3** teve tempos de extração extremamente elevados (superiores a 10.000 segundos em muitos casos), indicando um desempenho muito ineficiente ou até disfuncional.

📌 **Conclusão:**  
✅ Melhor: `VGGish`  
❌ Pior: `OpenL3`

---

#### 2️⃣ Tempo de Inserção no Milvus

- **VGGish** e **YAMNet** foram os mais rápidos na inserção.
- **OpenL3** e **AST** tiveram os maiores tempos de inserção, possivelmente devido à dimensão dos embeddings.

📌 **Conclusão:**  
✅ Melhor: `VGGish`  
❌ Pior: `OpenL3` / `AST`

---

#### 3️⃣ Boxplot do Tempo de Extração

- O boxplot confirmou a **grande dispersão e presença de outliers extremos** no `OpenL3`.
- `VGGish` e `Wav2Vec2` apresentaram baixa variância, sendo estáveis e previsíveis.

📌 **Conclusão:**  
✅ Melhor: `VGGish`  
❌ Pior: `OpenL3`

---

#### 4️⃣ Boxplot do Tempo de Inserção

- **OpenL3** teve vários outliers e distribuição mais espalhada.
- **VGGish**, **YAMNet** e **CLAP** mostraram estabilidade e baixa dispersão.

📌 **Conclusão:**  
✅ Melhor: `YAMNet` / `VGGish`  
❌ Pior: `OpenL3`

---

#### 5️⃣ Tamanho do Áudio vs Tempo de Extração

- **OpenL3** demonstrou valores fora da escala e sem correlação aparente com o tamanho.
- Os restantes modelos escalaram bem ou não foram fortemente afetados.

📌 **Conclusão:**  
✅ Melhor: `VGGish`, `CLAP`, `AST`  
❌ Pior: `OpenL3`

---

#### 6️⃣ Tamanho do Áudio vs Tempo de Inserção

- **VGGish**, **Wav2Vec2**, **YAMNet** e **CLAP** apresentaram tempos consistentes.
- **OpenL3** teve picos dispersos, com comportamento errático.

📌 **Conclusão:**  
✅ Melhor: `VGGish` / `CLAP`  
❌ Pior: `OpenL3`

---

### 📈 Tabela Comparativa Final

| Métrica                         | ✅ Melhor Modelo | ❌ Pior Modelo |
|--------------------------------|------------------|----------------|
| Tempo de Extração              | VGGish           | OpenL3         |
| Tempo de Inserção              | VGGish           | OpenL3         |
| Estabilidade (Boxplots)        | VGGish           | OpenL3         |
| Escalabilidade (vs Tamanho)    | VGGish / CLAP    | OpenL3         |
| Consistência Geral             | VGGish           | OpenL3         |

---

### 🧠 Conclusão

- O modelo **VGGish** demonstrou ser o mais eficiente, rápido, estável e previsível em todos os cenários testados.
- O modelo **OpenL3**, embora poderoso em teoria por ser multimodal, revelou-se impraticável para uso real em bases de dados vetoriais devido ao seu desempenho extremamente fraco, instável e com altos tempos de execução.

> Para aplicações com requisitos de performance, tempo real ou execução em larga escala, **VGGish** é a melhor escolha entre os modelos testados neste benchmark.


## 🔍 Interpretação Geral dos Resultados

Os resultados obtidos refletem as diferenças nas arquiteturas e complexidade dos modelos de embeddings de áudio utilizados.

---

### 🏆 VGGish – Melhor desempenho geral

- **Rápido, leve e estável**.
- Usa espectrogramas log-Mel e redes convolucionais simples.
- Embeddings pequenos (128 dimensões), o que acelera a inserção no Milvus.
- Ideal para aplicações com muitos dados ou tempo real.

---

### ❌ OpenL3 – Pior desempenho geral

- **Extremamente lento e instável**, com tempos de extração muito elevados.
- Arquitetura multimodal e pesada, com alto custo computacional.
- Embeddings grandes (512+ dimensões) e inferência demorada.
- Inadequado para aplicações práticas com muitos ficheiros.

---

### Outros modelos:

- **YAMNet**: também leve e estável; boa alternativa ao VGGish.
- **CLAP** e **AST**: mais pesados, mas geram embeddings de qualidade — bons para tarefas complexas.
- **Wav2Vec2**: equilibrado, mas mais orientado para voz.

---

### 📌 Conclusão

- O desempenho observado está diretamente ligado à **eficiência da arquitetura**, **tamanho dos embeddings** e **nível de pré-processamento**.
- **VGGish** é o modelo mais indicado para sistemas escaláveis e rápidos.  
- **OpenL3** deve ser evitado neste tipo de tarefa, devido à sua ineficiência.

A escolha do modelo deve equilibrar **qualidade dos embeddings** com **performance computacional**, dependendo dos objetivos do projeto.


### Evolução dos Recursos Computacionais

Nesta secção analisamos a evolução dos recursos computacionais utilizados durante o processo de extração de embeddings e inserção na base de dados vetorial. Monitorizamos duas métricas principais:

- **`cpu_percent_after`**: percentagem de utilização da CPU após cada processamento;
- **`ram_percent_after`**: percentagem de memória RAM utilizada no sistema após cada iteração.

---

### 📈 O que estamos a medir

Para cada ficheiro de áudio processado por cada modelo, medimos o uso dos recursos **imediatamente após a execução da extração e inserção**. Isto permite perceber:

- Se o modelo tem um consumo constante e previsível;
- Se há picos de uso ou variações inesperadas;
- Como o sistema se comporta ao longo do tempo (carga acumulada ou estabilização).

---

### ✅ Comportamentos esperados e considerados “bons”

Um bom desempenho ao nível dos recursos apresenta as seguintes características:

- **Uso estável e moderado da CPU** (idealmente abaixo de 70%, sem grandes flutuações);
- **Consumo de RAM controlado**, sem crescimento contínuo (indicativo de vazamentos de memória);
- **Curvas suaves** nos gráficos de evolução, com poucas variações bruscas entre iterações;
- Ausência de **picos isolados** ou comportamento errático ao longo do tempo.

---

### 🚩 Sinais de alerta

Comportamentos considerados problemáticos incluem:

- **Picos súbitos ou irregulares de CPU**, o que pode indicar modelos mal otimizados ou sobrecarga do sistema;
- **Crescimento progressivo da RAM** sem recuperação (possível memory leak);
- Modelos que, mesmo com ficheiros pequenos, causam **uso elevado e variável de recursos**, o que prejudica a escalabilidade.

---

### 📌 Objetivo da análise

Este tipo de análise é essencial para avaliar a **viabilidade prática** de um modelo em ambientes de produção, onde:

- A infraestrutura pode ter limitações (ex: edge devices, servidores partilhados);
- A consistência do uso de recursos é tão importante quanto a precisão dos embeddings;
- É necessário prever o custo computacional à medida que a base de dados cresce.

---

No gráfico gerado com `plotly`, conseguimos observar estas tendências para cada modelo ao longo da iteração dos ficheiros. Idealmente, os modelos mais leves (ex: VGGish, YAMNet) apresentam curvas mais suaves, enquanto modelos mais complexos (ex: CLAP, OpenL3, AST) podem gerar maior variabilidade — o que deve ser ponderado na escolha final do modelo.


In [7]:
# 📈 Evolução dos recursos (CPU, RAM)
recursos = ['cpu_percent_after', 'ram_percent_after']
for recurso in recursos:
    fig = go.Figure()
    for modelo in df['modelo'].unique():
        subset = df[df['modelo'] == modelo]
        fig.add_trace(go.Scatter(x=subset.index, y=subset[recurso], mode='lines+markers', name=modelo))
    fig.update_layout(title=f'Evolução de {recurso.upper()}', xaxis_title='Index', yaxis_title=recurso)
    fig.show()

## 📈 Análise da Evolução dos Recursos Computacionais

Nesta secção analisamos a evolução do uso da **CPU** e da **memória RAM** ao longo do tempo para cada modelo de embeddings de áudio. Esta análise é essencial para entender a **eficiência e estabilidade** dos modelos sob carga contínua.

---

### 🔧 CPU – `cpu_percent_after`

![Evolução da CPU](attachment:file-X98q35Af81JiktppDzqbPa)

#### 🟦 `wav2vec2`
- Consumo de CPU entre 60% e 75%, com boa estabilidade.
- Oscilações pequenas e comportamento relativamente previsível.
- Considerado eficiente e estável.

#### 🟥 `vggish`
- Apresenta **picos constantes a 90–100%**, mas também valores a 0%, o que indica **uso intermitente da CPU**.
- Esse padrão pode dever-se a espera por I/O ou gargalos no carregamento do modelo.
- Apesar de bom desempenho geral, o comportamento da CPU sugere **flutuação intensa**.

#### 🟩 `openl3`
- Uso de CPU extremamente elevado, próximo dos **100% em praticamente todas as iterações**.
- Sem variações, mas com **custo computacional máximo**.
- Comportamento esperado para um modelo muito pesado, mas **pouco eficiente**.

#### 🟪 `yamnet`
- Consumo moderado entre 20% e 60%, com **oscilações suaves**.
- Apresenta comportamento estável e previsível.
- Um dos mais eficientes em termos de uso de CPU.

#### 🟧 `clap`
- Variações acentuadas, com CPU entre 0% e 95%.
- Oscilações indicam alternância entre carga de processamento e espera.
- Padrão semelhante ao VGGish, mas com picos mais frequentes.

#### 🟦 `ast`
- Uso contínuo da CPU, com média alta (acima de 70%) e flutuação mais controlada.
- Estável, mas com custo computacional significativo.

---

### 💾 RAM – `ram_percent_after`

![Evolução da RAM](attachment:file-G8S7gsPCsCLvP6FZevbfLi)

#### 🟦 `wav2vec2`
- Consumo entre 81% e 84%, com variação suave.
- Comportamento previsível e eficiente.

#### 🟥 `vggish`
- Uso de RAM entre 84% e 86%, relativamente estável.
- Levemente mais exigente que Wav2Vec2.

#### 🟩 `openl3`
- Maior variabilidade no uso da RAM, oscilando de 78% a 85%.
- Este comportamento pode indicar **ineficiências na gestão de memória** ou pré-processamento mais pesado.
- Provável ausência de libertação imediata da memória.

#### 🟪 `yamnet`
- Uso da RAM crescente ao longo da execução.
- Possível **acumulação de carga** sem libertação de memória entre iterações.
- Mesmo assim, ainda aceitável em contextos com boa RAM disponível.

#### 🟧 `clap`
- Consumo estável, mas elevado (~86%), com tendência ascendente.
- Comportamento consistente com um modelo grande, mas não caótico.

#### 🟦 `ast`
- Estabilidade razoável no uso da RAM, mas com valores altos (~86%).
- Utilização elevada, mas sem crescimento contínuo (o que é positivo).

---

### ✅ Conclusões

| Modelo    | CPU: Estável | CPU: Alta | RAM: Estável | RAM: Uso Elevado | Observações                          |
|-----------|--------------|-----------|---------------|------------------|--------------------------------------|
| VGGish    | ❌           | ✅        | ✅            | ✅               | Uso intermitente, picos de CPU       |
| Wav2Vec2  | ✅           | ⚠️        | ✅            | ✅               | Estável e eficiente                  |
| OpenL3    | ✅           | ✅        | ❌            | ✅               | Altíssimo custo de CPU e RAM         |
| YAMNet    | ✅           | ❌        | ⚠️            | ⚠️               | Leve e eficiente, mas RAM crescente  |
| CLAP      | ❌           | ✅        | ✅            | ✅               | Variações intensas, mas previsível   |
| AST       | ✅           | ✅        | ✅            | ✅               | Carga constante e alta, mas estável  |

---

### 🧠 Considerações Finais

- **Modelos como YAMNet e Wav2Vec2** mostram o melhor equilíbrio entre performance e uso de recursos.
- **OpenL3** utiliza consistentemente **quase todos os recursos da máquina**, o que o torna **ineficiente para produção**.
- **CLAP e AST** são pesados, mas mantêm padrões estáveis, o que pode ser aceitável em contextos com infraestrutura robusta.

A escolha ideal depende dos requisitos da aplicação:  
→ Para **baixa carga e tempo real**, **YAMNet ou VGGish** são os mais indicados.  
→ Para **análises profundas ou embeddings ricos**, **AST** pode ser preferido — desde que haja recurso disponível.


# Ranking Modelos

## Explicação do Sistema de Ranking dos Modelos de Embeddings
O sistema de ranking implementado no notebook oferece uma visão comparativa detalhada sobre o desempenho dos diferentes modelos de embeddings de áudio. Vamos explicar o significado dos valores apresentados e como interpretá-los:

Rankings Individuais por Métrica:
- Tempo de Extração (s)
Representa quanto tempo cada modelo leva para processar o áudio e gerar o embedding
Valores menores são melhores (indicam processamento mais rápido)
Um modelo com tempo de extração baixo é ideal para aplicações em tempo real ou processamento em lote de grandes coleções de áudio
- Tempo de Inserção (s)
Mede quanto tempo leva para armazenar o embedding na base de dados Milvus
Valores menores são melhores
Geralmente menos crítico que o tempo de extração, mas importante para fluxos de ingestão de dados em grande escala
- Pico de Memória (MB)
Indica a quantidade máxima de memória RAM utilizada durante o processamento
Valores menores são melhores (menos recursos consumidos)
Crítico para ambientes com recursos limitados ou aplicações que precisam processar muitos arquivos simultaneamente
- Uso de CPU (%)
Porcentagem de utilização da CPU durante o processamento
Valores menores são melhores
Modelos com menor uso de CPU deixam mais recursos disponíveis para outras tarefas
- Uso de RAM (%)
Percentual total de memória do sistema utilizada
Valores menores são melhores
Diferente do "Pico de Memória" que é específico do processo
- Dimensionalidade
Tamanho do vetor de embedding gerado
Não é classificado como "melhor" ou "pior" diretamente

Embeddings maiores (maior dimensionalidade) podem capturar mais informações, mas:
Consomem mais espaço de armazenamento
Podem ser mais lentos para pesquisa
Podem exigir mais recursos computacionais
Ranking Geral
Pontuação (0-100)
Representa uma avaliação holística do desempenho do modelo
Valores maiores são melhores (mais próximo de 100 = desempenho ideal)
Calculada usando uma média ponderada normalizada das métricas principais
Pesos das Métricas no Ranking Geral
O ranking geral usa a seguinte distribuição de pesos:

Tempo de extração: 35% (mais peso devido à sua importância crítica)
Pico de memória: 25% (segundo mais importante)
Uso de CPU: 20% (impacto moderado)
Tempo de inserção: 15% (menos crítico que a extração)
Uso de RAM total: 5% (menor peso pois depende de outros fatores do sistema)
Processo de Normalização
Para criar um ranking justo, o sistema:

Normaliza cada métrica para uma escala de 0-1 (onde 0 é o melhor valor)
Aplica os pesos definidos para cada métrica
Inverte a escala final para que valores maiores (próximos de 100) sejam melhores
Interpretação dos Resultados
Primeiro lugar (ouro): Representa o modelo com melhor equilíbrio entre velocidade, eficiência de recursos e desempenho
Segundo lugar (prata): Bom desempenho geral, possível alternativa ao primeiro colocado
Terceiro lugar (bronze): Desempenho aceitável, pode ser útil em cenários específicos
Demais posições: Podem ser adequadas para casos de uso específicos, dependendo da prioridade (velocidade vs. qualidade vs. recursos)
É importante notar que o ranking geral oferece uma visão equilibrada, mas a escolha do melhor modelo ainda depende das prioridades específicas do seu projeto. Por exemplo, se você tem restrições severas de memória, pode preferir um modelo que não está no topo do ranking geral, mas que tem excelente desempenho na métrica de consumo de memória.

In [8]:
# 🏆 Ranking dos modelos por diferentes métricas
def create_ranking_table(df, metric_column, metric_name, lower_is_better=True):
    """
    Cria uma tabela de classificação dos modelos baseada numa métrica específica.
    
    Args:
        df: DataFrame com os resultados do benchmark
        metric_column: Nome da coluna da métrica a ser analisada
        metric_name: Nome amigável da métrica para exibição
        lower_is_better: Se True, valores menores são melhores (ex: tempo, memória)
    
    Returns:
        DataFrame com o ranking dos modelos
    """
    # Calcular média e desvio padrão para cada modelo
    ranking = df.groupby('modelo')[metric_column].agg(['mean', 'std', 'min', 'max']).reset_index()
    
    # Renomear colunas
    ranking.columns = ['Modelo', f'Média {metric_name}', f'Desvio Padrão', f'Mínimo', f'Máximo']
    
    # Ordenar resultados (crescente se lower_is_better=True, decrescente caso contrário)
    ranking = ranking.sort_values(by=f'Média {metric_name}', ascending=lower_is_better)
    
    # Adicionar coluna de ranking
    ranking.insert(0, 'Posição', range(1, len(ranking) + 1))
    
    return ranking

# Criar rankings para diferentes métricas
try:
    print("🏆 Rankings dos Modelos de Embeddings de Áudio")
    
    # Definir as métricas e seus nomes amigáveis
    metrics = [
        ('tempo_extracao', 'Tempo de Extração (s)', True),
        ('tempo_insercao', 'Tempo de Inserção (s)', True),
        ('memoria_peak_mb', 'Pico de Memória (MB)', True),
        ('cpu_percent_after', 'Uso de CPU (%)', True),
        ('ram_percent_after', 'Uso de RAM (%)', True),
        ('gpu_usage', 'Uso de GPU (%)', True),
        ('dimensao', 'Dimensionalidade', None)  # None porque não é melhor nem pior
    ]
    
    # Criar abas para os diferentes rankings
    tab_contents = [m[1] for m in metrics]
    children = []
    
    for metric_col, metric_name, lower_is_better in metrics:
        if metric_col in df.columns:
            # Criar tabela de ranking
            if lower_is_better is None:  # Para dimensionalidade, só mostramos os valores sem julgar
                ranking = df.groupby('modelo')[metric_col].first().reset_index()
                ranking.columns = ['Modelo', metric_name]
                ranking = ranking.sort_values(by=metric_name)
                ranking.insert(0, 'Posição', range(1, len(ranking) + 1))
            else:
                ranking = create_ranking_table(df, metric_col, metric_name, lower_is_better)
            
            # Criar visualização de tabela colorida com Plotly
            fig = go.Figure(data=[go.Table(
                header=dict(
                    values=list(ranking.columns),
                    fill_color='royalblue',
                    font=dict(color='white', size=12),
                    align='left'
                ),
                cells=dict(
                    values=[ranking[col] for col in ranking.columns],
                    fill_color=[['lightcyan' if i % 2 == 0 else 'white' for i in range(len(ranking))]],
                    align='left',
                    font_size=11,
                    height=25
                )
            )])
            
            fig.update_layout(
                title=f"Ranking por {metric_name}",
                height=400,
                margin=dict(l=20, r=20, t=40, b=20)
            )
            
            # Adicionar à aba
            out = widgets.Output()
            with out:
                display(fig)
            children.append(out)
        else:
            # Se a métrica não existe, mostrar mensagem
            out = widgets.Output()
            with out:
                print(f"⚠️ Métrica '{metric_name}' não disponível nos dados do benchmark.")
            children.append(out)
    
    # Criar widget de abas para os rankings
    ranking_tab = widgets.Tab(children=children)
    for i in range(len(tab_contents)):
        ranking_tab.set_title(i, tab_contents[i])
    
    # Mostrar widget
    display(ranking_tab)
    
    # Criar um ranking geral (média ponderada das principais métricas)
    print("\n🏆 Ranking Geral dos Modelos")
    
    # Definir pesos para as métricas principais (ajuste conforme necessidade do projeto)
    weights = {
        'tempo_extracao': 0.35,  # Tempo de extração é muito importante
        'memoria_peak_mb': 0.25,  # Uso de memória é importante
        'cpu_percent_after': 0.20,  # Uso de CPU tem peso médio
        'tempo_insercao': 0.15,  # Tempo de inserção é menos importante
        'ram_percent_after': 0.05  # RAM total tem menos peso
    }
    
    # Verificar quais métricas estão disponíveis
    available_metrics = [m for m in weights.keys() if m in df.columns]
    
    if len(available_metrics) >= 3:  # Exigir pelo menos 3 métricas para um ranking geral adequado
        # Normalizar os pesos para as métricas disponíveis
        total_weight = sum(weights[m] for m in available_metrics)
        normalized_weights = {m: weights[m]/total_weight for m in available_metrics}
        
        # Calcular médias para cada modelo e métrica
        model_metrics = {}
        for model in df['modelo'].unique():
            model_data = df[df['modelo'] == model]
            model_metrics[model] = {}
            
            for metric in available_metrics:
                model_metrics[model][metric] = model_data[metric].mean()
        
        # Normalizar valores entre 0-1 (onde 0 é melhor, 1 é pior)
        normalized_metrics = {}
        for metric in available_metrics:
            min_val = min(model_metrics[m][metric] for m in model_metrics)
            max_val = max(model_metrics[m][metric] for m in model_metrics)
            range_val = max_val - min_val
            
            if range_val > 0:  # Evitar divisão por zero
                for model in model_metrics:
                    if metric not in normalized_metrics:
                        normalized_metrics[metric] = {}
                    normalized_metrics[metric][model] = (model_metrics[model][metric] - min_val) / range_val
            else:
                # Se todos os valores forem iguais, atribuir 0.5 (neutro)
                for model in model_metrics:
                    if metric not in normalized_metrics:
                        normalized_metrics[metric] = {}
                    normalized_metrics[metric][model] = 0.5
        
        # Calcular pontuação final (ponderada)
        final_scores = {}
        for model in model_metrics:
            score = 0
            for metric in available_metrics:
                score += normalized_metrics[metric][model] * normalized_weights[metric]
            final_scores[model] = score
        
        # Criar DataFrame com ranking final
        final_ranking = pd.DataFrame({
            'Modelo': list(final_scores.keys()),
            'Pontuação Final': list(final_scores.values())
        })
        
        # Calcular pontuação de 0-100 (inverter para que maior seja melhor)
        final_ranking['Pontuação (0-100)'] = 100 * (1 - final_ranking['Pontuação Final'])
        
        # Ordenar por pontuação (maior é melhor)
        final_ranking = final_ranking.sort_values('Pontuação (0-100)', ascending=False)
        final_ranking.insert(0, 'Posição', range(1, len(final_ranking) + 1))
        
        # Criar visualização de tabela colorida para o ranking final
        fig = go.Figure(data=[go.Table(
            header=dict(
                values=['Posição', 'Modelo', 'Pontuação (0-100)'],
                fill_color='darkblue',
                font=dict(color='white', size=14),
                align='center'
            ),
            cells=dict(
                values=[
                    final_ranking['Posição'],
                    final_ranking['Modelo'],
                    final_ranking['Pontuação (0-100)'].round(2)
                ],
                fill_color=[
                    ['gold' if pos == 1 else 'silver' if pos == 2 else 'lightcoral' if pos == 3 
                     else 'white' for pos in final_ranking['Posição']]
                ],
                align='center',
                font_size=13,
                height=30
            )
        )])
        
        fig.update_layout(
            title="🏆 Ranking Geral dos Modelos de Embeddings de Áudio",
            height=400,
            margin=dict(l=20, r=20, t=50, b=20)
        )
        
        display(fig)
        
        # Criar gráfico de barras para o ranking final
        bar_fig = px.bar(
            final_ranking, 
            x='Modelo', 
            y='Pontuação (0-100)', 
            title='Comparação Geral dos Modelos', 
            color='Pontuação (0-100)',
            color_continuous_scale=px.colors.sequential.Viridis,
            text='Pontuação (0-100)'
        )
        
        bar_fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')
        bar_fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
        
        display(bar_fig)
        
    else:
        print("⚠️ Dados insuficientes para gerar um ranking geral significativo.")

except Exception as e:
    import traceback
    print(f"❌ Erro ao gerar ranking dos modelos: {e}")
    traceback.print_exc()

🏆 Rankings dos Modelos de Embeddings de Áudio



🏆 Ranking Geral dos Modelos


## 🏁 Conclusão Final: Por que o YAMNet foi o modelo vencedor

A escolha do **YAMNet** como o modelo com melhor desempenho geral neste benchmark justifica-se por um equilíbrio notável entre **eficiência, estabilidade e qualidade técnica**, tendo em conta os seguintes fatores:

---

### ✅ 1. Desempenho técnico consistente
- **Tempos de extração e inserção baixos**, tornando-o adequado para cenários em tempo real.
- Apresentou **boa estabilidade nos boxplots**, com dispersão controlada e poucos outliers.
- Embora não tenha sido o mais rápido em todas as métricas, demonstrou **regularidade superior** aos demais.

---

### 💾 2. Uso eficiente dos recursos
- **Baixo uso de CPU**, mantendo-se geralmente entre os 20% e 60%, o que demonstra eficiência energética.
- Apesar de uma **ligera tendência de crescimento no uso da RAM**, manteve-se dentro de limites aceitáveis e não indicou vazamentos de memória graves.
- Comportamento previsível e controlado mesmo em execução prolongada.

---

### ⚖️ 3. Excelente relação performance/custo
- Comparado com modelos mais pesados como **CLAP** ou **AST**, o YAMNet oferece um **ótimo compromisso entre qualidade de embedding e custo computacional**.
- Produz embeddings ricos (1024 dimensões) com **menos impacto nos recursos do sistema**.

---

### 📈 4. Elevada pontuação no ranking final
- Alcançou a **pontuação mais alta (97.96/100)** no ranking geral, que agregou múltiplas métricas de desempenho, consumo e escalabilidade.
- Superou inclusive modelos como o **VGGish** (mais leve) e o **CLAP** (mais semântico), mostrando-se mais robusto no conjunto.

---

### 🧠 5. Adequação prática e versatilidade
- É um modelo pronto para tarefas de classificação e similaridade sonora, treinado em larga escala com o **AudioSet**.
- Integra-se facilmente em pipelines de pré-processamento com frameworks como TensorFlow Hub.
- Ideal para **bases de dados vetoriais** onde se busca bom desempenho com escalabilidade.

---

### 🥇 Conclusão

> O **YAMNet** foi escolhido como o melhor modelo de embeddings de áudio neste benchmark não por ser o mais rápido ou o mais leve, mas por oferecer **a melhor performance global**: estável, eficiente e com baixo custo computacional — uma escolha segura e versátil para aplicações reais.


## 📈 Análise de Escalabilidade dos Modelos

A escalabilidade é um dos fatores mais importantes ao escolher um modelo de embeddings para grandes bases de dados. Um modelo eficiente não deve apenas ter bom desempenho em cenários pequenos, mas também **manter a sua eficiência relativa à medida que o volume de dados cresce**.

---

### 🔍 Como foi feita a análise

Foram realizados **vários benchmarks independentes**, variando o número de ficheiros processados:  
`[5, 25, 50, 100, 250]`

Para cada uma destas escalas, foram medidos os seguintes indicadores:

- **Tempo médio de extração de embeddings**;
- **Tempo médio de inserção no Milvus**;
- **Pico médio de uso de memória (RAM)**;
- **Uso médio de CPU (%)**;
- **Ranking geral baseado em desempenho e eficiência combinados**.

---

### 🧪 Estratégia utilizada

1. **Carregamento automático dos benchmarks**: os dados foram estruturados para serem facilmente carregados e analisados por escala.
2. **Visualizações interativas**: foram criados gráficos comparativos para observar o comportamento de cada modelo com o aumento do número de ficheiros.
3. **Cálculo da tendência de crescimento (slope)**: foi estimada a **taxa de crescimento do tempo de extração** para cada modelo, indicando se ele cresce linear, exponencial ou de forma controlada com o volume de dados.
4. **Estabilidade de ranking**: foi analisada a **posição relativa dos modelos em diferentes tamanhos de benchmark**, com um heatmap mostrando quais modelos mantêm performance consistente.

---

### ✅ Comportamento esperado para um modelo escalável

Um modelo considerado **escalável** deve:

- Ter uma **taxa de crescimento baixa** na extração de embeddings;
- Manter **uso de CPU e RAM estáveis** mesmo com mais ficheiros;
- Preservar uma posição elevada no ranking à medida que o volume aumenta;
- Não degradar significativamente o tempo de inserção ou processamento.

---

### 📊 Ferramentas utilizadas

- **Gráficos de linha**: mostram como cada métrica varia com o número de ficheiros.
- **Tabelas interativas e heatmaps**: revelam como os rankings se alteram com a escala.
- **Cálculo de slope (taxa de crescimento)**: estima qual modelo é mais sensível ao aumento de dados.

---

### 🏁 Conclusão da análise de escalabilidade

Esta abordagem permite identificar **quais modelos são apropriados para ambientes de produção com crescimento contínuo de dados**, ajudando a:

- Antecipar problemas de performance em larga escala;
- Escolher modelos que se comportem bem tanto em fase de prototipagem quanto em produção;
- Tomar decisões baseadas não apenas no desempenho atual, mas também na **projeção futura do custo computacional**.

> ⚠️ Mesmo modelos rápidos com poucos ficheiros podem tornar-se **ineficientes ou inviáveis** se não escalarem bem. Esta análise garante que essa escolha é feita de forma informada.


In [9]:
# 📊 Comparação de benchmarks com diferentes quantidades de ficheiros
from IPython.display import display, clear_output

def carregar_multiplos_benchmarks():
    """Carrega todos os benchmarks disponíveis em uma estrutura unificada"""
    tamanhos = [5, 25, 50, 100, 250]
    dataframes = {}
    
    for tamanho in tamanhos:
        try:
            caminho = f'benchmarks/benchmark_{tamanho}/resultados_benchmark.csv'
            df = pd.read_csv(caminho)
            df['embedding'] = df['embedding'].apply(ast.literal_eval)
            df['tamanho_benchmark'] = tamanho  # Adiciona coluna identificando o tamanho
            dataframes[tamanho] = df
            print(f"✅ Benchmark com {tamanho} ficheiros carregado: {len(df)} registros")
        except FileNotFoundError:
            print(f"⚠️ Benchmark com {tamanho} ficheiros não encontrado")
        except Exception as e:
            print(f"❌ Erro ao carregar benchmark com {tamanho} ficheiros: {e}")
    
    return dataframes

# Função para exibir visualizações de um benchmark específico
def display_benchmark(df):
    """Exibe visualizações para um benchmark específico"""
    if df is None or df.empty:
        print("Sem dados para exibir")
        return
    
    # Limpar saída anterior
    clear_output(wait=True)
    
    # Mostrar informações básicas
    print(f"## Benchmark com {df['tamanho_benchmark'].iloc[0]} ficheiros")
    print(f"- Total de registros: {len(df)}")
    print(f"- Modelos disponíveis: {', '.join(df['modelo'].unique())}")
    
    # Visualizações principais
    fig1 = px.bar(df, x='modelo', y='tempo_extracao', color='modelo', 
                 title=f'Tempo de Extração por Modelo (Base: {df["tamanho_benchmark"].iloc[0]} ficheiros)')
    fig2 = px.bar(df, x='modelo', y='tempo_insercao', color='modelo', 
                 title=f'Tempo de Inserção por Modelo (Base: {df["tamanho_benchmark"].iloc[0]} ficheiros)')
    
    display(fig1)
    display(fig2)
    
    # Mostrar estatísticas
    stats = df.groupby('modelo')[['tempo_extracao', 'tempo_insercao']].agg(['mean', 'std']).reset_index()
    display(stats)

# Carregar todos os benchmarks
benchmarks = carregar_multiplos_benchmarks()

# Interface para seleção do benchmark a visualizar
tamanhos_disponiveis = list(benchmarks.keys())
if tamanhos_disponiveis:
    # Definir a função de atualização de visualização
    def atualizar_visualizacao(change):
        tamanho_selecionado = change['new']
        if tamanho_selecionado in benchmarks:
            display_benchmark(benchmarks[tamanho_selecionado])
    
    # Criar o seletor
    selector = widgets.Dropdown(
        options=tamanhos_disponiveis,
        description='Benchmark:',
        value=tamanhos_disponiveis[0]
    )
    
    selector.observe(atualizar_visualizacao, names='value')
    display(selector)
    
    # Mostrar o benchmark inicial
    display_benchmark(benchmarks[tamanhos_disponiveis[0]])
else:
    print("Nenhum benchmark encontrado.")

# Análise de escalabilidade
if len(benchmarks) >= 2:
    print("\n# Análise de Escalabilidade")
    print("Observe como cada modelo se comporta com o aumento do número de ficheiros:")
    
    # 1. Gráfico de evolução de tempo de extração com o aumento do número de arquivos
    metricas_escalabilidade = [
        ('tempo_extracao', 'Tempo Médio de Extração (s)'),
        ('tempo_insercao', 'Tempo Médio de Inserção (s)'),
        ('memoria_peak_mb', 'Pico Médio de Memória (MB)'),
        ('cpu_percent_after', 'Uso Médio de CPU (%)')
    ]
    
    tab_contents_escala = [m[1] for m in metricas_escalabilidade]
    children_escala = []
    
    for metrica, titulo in metricas_escalabilidade:
        # Criar figura
        fig_escala = go.Figure()
        
        # Verificar se a métrica existe em pelo menos um dataframe
        if not any(metrica in df.columns for df in benchmarks.values()):
            out = widgets.Output()
            with out:
                print(f"Métrica '{titulo}' não disponível nos dados")
            children_escala.append(out)
            continue
        
        # Para cada modelo, coletar dados de todos os benchmarks
        all_models = set()
        for df in benchmarks.values():
            if not df.empty:
                all_models.update(df['modelo'].unique())
        
        for modelo in all_models:
            x_vals = []  # Tamanhos dos benchmarks
            y_vals = []  # Valores médios da métrica
            
            for tamanho, df in benchmarks.items():
                if not df.empty and modelo in df['modelo'].unique() and metrica in df.columns:
                    x_vals.append(tamanho)
                    y_vals.append(df[df['modelo'] == modelo][metrica].mean())
            
            # Ordenar por tamanho (para garantir que a linha seja traçada corretamente)
            pontos = sorted(zip(x_vals, y_vals))
            if pontos:
                x_vals, y_vals = zip(*pontos)
                fig_escala.add_trace(go.Scatter(
                    x=x_vals, 
                    y=y_vals, 
                    mode='lines+markers', 
                    name=modelo
                ))
        
        fig_escala.update_layout(
            title=f'Escalabilidade: {titulo} por Tamanho da Base',
            xaxis_title='Número de Ficheiros',
            yaxis_title=titulo,
            hovermode='closest'
        )
        
        # Adicionar à aba
        out = widgets.Output()
        with out:
            display(fig_escala)
        children_escala.append(out)
    
    # Criar widget de abas para os gráficos de escalabilidade
    escalabilidade_tab = widgets.Tab(children=children_escala)
    for i in range(len(tab_contents_escala)):
        escalabilidade_tab.set_title(i, tab_contents_escala[i])
    
    display(escalabilidade_tab)
    
    # 2. Comparação de rankings entre benchmarks
    print("\n## Evolução dos Rankings com a Escala")
    
    # Extrair rankings para cada tamanho
    rankings_por_tamanho = {}
    for tamanho, df in benchmarks.items():
        if not df.empty:
            # Usar algoritmo de ranking simplificado
            rankings = {}
            for modelo in df['modelo'].unique():
                # Cálculo do ranking (simplificado)
                metricas = {
                    'tempo_extracao': df[df['modelo'] == modelo]['tempo_extracao'].mean(),
                    'memoria_peak_mb': df[df['modelo'] == modelo]['memoria_peak_mb'].mean() if 'memoria_peak_mb' in df.columns else None,
                    'cpu_percent_after': df[df['modelo'] == modelo]['cpu_percent_after'].mean() if 'cpu_percent_after' in df.columns else None,
                    'tempo_insercao': df[df['modelo'] == modelo]['tempo_insercao'].mean() if 'tempo_insercao' in df.columns else None
                }
                # Pontuação simples (quanto menor, melhor)
                pontuacao = sum(v for v in metricas.values() if v is not None) 
                rankings[modelo] = pontuacao
            
            # Ordenar do melhor para o pior
            rankings_ordenados = {k: v for k, v in sorted(rankings.items(), key=lambda item: item[1])}
            rankings_por_tamanho[tamanho] = rankings_ordenados
    
    # Visualizar como os rankings mudam
    # Visualizar como os rankings mudam
if rankings_por_tamanho:
    # Criar tabela de evolução de posições
    modelos = set()
    for r in rankings_por_tamanho.values():
        modelos.update(r.keys())
    
    # Convert set to list for pandas DataFrame index
    modelos = sorted(list(modelos))
    
    tabela_posicoes = pd.DataFrame(index=modelos)
    
    for tamanho, ranking in sorted(rankings_por_tamanho.items()):
        # Converte o ranking em posições (1º, 2º, 3º...)
        posicoes = {modelo: pos+1 for pos, modelo in enumerate(ranking.keys())}
        tabela_posicoes[f'{tamanho} ficheiros'] = pd.Series(posicoes)
        
        # Mostrar tabela colorida
        fig_ranking = go.Figure(data=[go.Table(
            header=dict(
                values=['Modelo'] + [f'{t} ficheiros' for t in sorted(rankings_por_tamanho.keys())],
                fill_color='royalblue',
                font=dict(color='white', size=12),
                align='center'
            ),
            cells=dict(
                values=[tabela_posicoes.index] + [tabela_posicoes[col] for col in tabela_posicoes.columns],
                fill_color='lightcyan',
                align='center'
            )
        )])
        
        fig_ranking.update_layout(
            title="Evolução das Posições no Ranking por Escala",
            height=400,
            margin=dict(l=20, r=20, t=50, b=20)
        )
        
        display(fig_ranking)
        
        # Heatmap de estabilidade do ranking
        heatmap_data = tabela_posicoes.copy()
        
        # Normalizar valores para visualização (posição 1 = melhor = 1.0, posições piores = valores menores)
        for col in heatmap_data.columns:
            max_pos = heatmap_data[col].max()
            if pd.notna(max_pos) and max_pos > 1:
                heatmap_data[col] = 1 - (heatmap_data[col] - 1) / max_pos
        
        fig_heatmap = px.imshow(
            heatmap_data,
            labels=dict(x="Tamanho do Benchmark", y="Modelo", color="Posição Relativa"),
            x=heatmap_data.columns,
            y=heatmap_data.index,
            color_continuous_scale='viridis',
            title="Estabilidade dos Rankings com Escala"
        )
        
        display(fig_heatmap)
        
        # Análise de eficiência com escala
        print("\n## Eficiência com Escala")
        print("Esta análise mostra quais modelos mantêm bom desempenho mesmo com grandes volumes de dados:")
        
        # Calcular a tendência de cada modelo em tempo de extração com aumento da escala
        slopes = {}
        for modelo in modelos:
            x_vals = []
            y_vals = []
            for tamanho, df in benchmarks.items():
                if not df.empty and modelo in df['modelo'].unique() and 'tempo_extracao' in df.columns:
                    x_vals.append(tamanho)
                    y_vals.append(df[df['modelo'] == modelo]['tempo_extracao'].mean())
            
            if len(x_vals) >= 2:
                # Calcular tendência (aproximação simples)
                if max(x_vals) > min(x_vals):
                    slope = (max(y_vals) - min(y_vals)) / (max(x_vals) - min(x_vals))
                    slopes[modelo] = slope
        
        if slopes:
            slopes_df = pd.DataFrame({
                'Modelo': list(slopes.keys()),
                'Taxa de Crescimento': list(slopes.values())
            })
            
            slopes_df = slopes_df.sort_values('Taxa de Crescimento')
            
            fig_slopes = px.bar(
                slopes_df, 
                x='Modelo', 
                y='Taxa de Crescimento',
                title='Taxa de Crescimento do Tempo com Aumento da Base',
                color='Taxa de Crescimento',
                color_continuous_scale='RdYlGn_r'  # Vermelho = pior, Verde = melhor
            )
            
            display(fig_slopes)
            
            print("\nModelos com menor taxa de crescimento são mais escaláveis, pois seu tempo de processamento")
            print("aumenta mais lentamente com o crescimento do volume de dados.")
            
            # Adicionar interpretação dos resultados
            melhor_modelo = slopes_df.iloc[0]['Modelo']
            pior_modelo = slopes_df.iloc[-1]['Modelo']
            
            print(f"\n### Interpretação dos Resultados de Escalabilidade")
            print(f"- **{melhor_modelo}** demonstra a melhor escalabilidade, com menor taxa de crescimento de tempo.")
            print(f"- **{pior_modelo}** apresenta a escalabilidade mais desafiadora, com maior aumento de tempo conforme o volume de dados cresce.")
            print("\nEscolha o modelo considerando não apenas o desempenho atual, mas também a projeção futura do volume de dados.")

## Benchmark com 5 ficheiros
- Total de registros: 30
- Modelos disponíveis: wav2vec2, vggish, openl3, yamnet, clap, ast


modelo tempo_extracao           tempo_insercao          
                      mean       std           mean       std
0       ast       0.838650  0.332128       0.158832  0.129398
1      clap       0.229738  0.202934       0.053357  0.012093
2    openl3      27.741236  2.404783       0.082798  0.043130
3    vggish       0.285770  0.154726       0.048521  0.019274
4  wav2vec2       1.871834  0.884459       0.184742  0.243238
5    yamnet       0.150472  0.124131       0.044936  0.005034


# Análise de Escalabilidade
Observe como cada modelo se comporta com o aumento do número de ficheiros:



## Evolução dos Rankings com a Escala



## Eficiência com Escala
Esta análise mostra quais modelos mantêm bom desempenho mesmo com grandes volumes de dados:


...


Modelos com menor taxa de crescimento são mais escaláveis, pois seu tempo de processamento
aumenta mais lentamente com o crescimento do volume de dados.

### Interpretação dos Resultados de Escalabilidade
- **vggish** demonstra a melhor escalabilidade, com menor taxa de crescimento de tempo.
- **openl3** apresenta a escalabilidade mais desafiadora, com maior aumento de tempo conforme o volume de dados cresce.

Escolha o modelo considerando não apenas o desempenho atual, mas também a projeção futura do volume de dados.



## Eficiência com Escala
Esta análise mostra quais modelos mantêm bom desempenho mesmo com grandes volumes de dados:



Modelos com menor taxa de crescimento são mais escaláveis, pois seu tempo de processamento
aumenta mais lentamente com o crescimento do volume de dados.

### Interpretação dos Resultados de Escalabilidade
- **vggish** demonstra a melhor escalabilidade, com menor taxa de crescimento de tempo.
- **openl3** apresenta a escalabilidade mais desafiadora, com maior aumento de tempo conforme o volume de dados cresce.

Escolha o modelo considerando não apenas o desempenho atual, mas também a projeção futura do volume de dados.



## Eficiência com Escala
Esta análise mostra quais modelos mantêm bom desempenho mesmo com grandes volumes de dados:



Modelos com menor taxa de crescimento são mais escaláveis, pois seu tempo de processamento
aumenta mais lentamente com o crescimento do volume de dados.

### Interpretação dos Resultados de Escalabilidade
- **vggish** demonstra a melhor escalabilidade, com menor taxa de crescimento de tempo.
- **openl3** apresenta a escalabilidade mais desafiadora, com maior aumento de tempo conforme o volume de dados cresce.

Escolha o modelo considerando não apenas o desempenho atual, mas também a projeção futura do volume de dados.


Benefícios desta Abordagem
Análise de Escalabilidade: Mostra como cada modelo se comporta à medida que o número de arquivos aumenta

Evolução dos Rankings: Permite ver se os modelos que são melhores em conjuntos pequenos mantêm sua posição com conjuntos maiores

Estabilidade de Desempenho: O heatmap visualiza quais modelos são mais consistentes em diferentes escalas

Análise de Eficiência: Identifica quais modelos têm melhor "crescimento" de tempo/recursos, indicando escalabilidade

Interface Interativa: Permite alternar facilmente entre diferentes tamanhos de benchmark

Interpretação dos Resultados
Taxa de Crescimento: Modelos com menor taxa indicam melhor escalabilidade
Estabilidade do Ranking: Modelos que mantêm cores consistentes no heatmap são mais previsíveis em diferentes escalas
Evolução das Métricas: Os gráficos de linha mostram se o crescimento é linear, exponencial ou irregular
Esta análise permitirá determinar qual modelo é mais adequado não apenas para o tamanho atual da sua base, mas também para o crescimento futuro, facilitando decisões estratégicas sobre qual embedding adotar em produção.

## 📈 Análise de Escalabilidade dos Modelos

Com base nas imagens dos gráficos de escalabilidade, realizamos uma análise detalhada do comportamento dos modelos face ao aumento do número de ficheiros (5, 25, 50, 100, 250). As principais métricas observadas foram: tempo médio de extração, tempo médio de inserção, pico médio de memória e uso médio de CPU.

---

### 🔹 1. Tempo Médio de Extração

- **openl3** apresenta claramente o pior desempenho nesta métrica, com tempo médio de extração que cresce significativamente com o número de ficheiros, atingindo mais de 45 segundos com 250 ficheiros.
- Todos os outros modelos mantêm tempos baixos e relativamente estáveis, com destaque para **yamnet**, que se mantém consistentemente abaixo de 1 segundo.
- **Conclusão**: *yamnet* e *vggish* demonstram excelente escalabilidade nesta métrica.

---

### 🔹 2. Tempo Médio de Inserção

- A maior parte dos modelos tem curvas suaves e estáveis, com tempos de inserção abaixo de 0.2s.
- **openl3** volta a destacar-se negativamente, com um aumento notável à medida que a base cresce.
- **clap** e **yamnet** mantêm tempos de inserção baixos e consistentes — ideal para bases vetoriais dinâmicas.

---

### 🔹 3. Pico Médio de Memória RAM

- Todos os modelos mostram comportamento muito estável em termos de uso de memória, independentemente da escala.
- Mesmo **openl3**, que teve mau desempenho noutras métricas, mantém uso de RAM relativamente constante.
- **yamnet** apresenta um dos menores picos médios de memória, o que reforça sua eficiência.

---

### 🔹 4. Uso Médio de CPU

- **openl3** mantém o uso de CPU consistentemente elevado (acima dos 90%), sugerindo uma alta carga de processamento.
- **yamnet**, por contraste, opera com uso de CPU bastante baixo e estável (~25%), o que é altamente desejável em ambientes de produção.
- **clap** e **ast** apresentam variações suaves, indicando bom controlo da carga mesmo com bases maiores.

---

### 🧠 Padrões Observados

| Modelo     | Tempo Extração | Inserção | RAM | CPU | Escalabilidade Geral |
|------------|----------------|----------|-----|-----|------------------------|
| **yamnet** | Excelente       | Muito boa | Baixa | Baixíssima | ⭐ Melhor |
| clap       | Boa            | Boa       | Baixa | Moderada | Boa |
| vggish     | Boa            | Excelente | Baixa | Alta     | Boa |
| wav2vec2   | Média          | Média     | Baixa | Alta     | Razoável |
| ast        | Média          | Média     | Baixa | Média    | Razoável |
| **openl3** | Fraca          | Fraca     | Constante mas alta | Altíssima | ❌ Fraca |

---

### ✅ Conclusão

A partir da análise de escalabilidade:

- **Yamnet** é o modelo com melhor comportamento geral. Combina baixo tempo de extração, baixo uso de CPU, baixo uso de memória e boa estabilidade com o aumento do volume de dados.
- **Openl3** mostra-se ineficiente e pouco escalável: é o único modelo que degrada visivelmente com a escala, sendo o menos recomendado.
- Outros modelos como **clap** e **vggish** apresentam desempenho sólido, mas com maiores exigências de CPU ou menos estabilidade com crescimento da base.

> Esta análise reforça a importância de não apenas olhar para a performance bruta de um modelo, mas sim para o seu comportamento em larga escala — fundamental em aplicações reais com milhares de ficheiros de áudio.


### Visualização dos Embeddings no Espaço

Para melhor compreender como os diferentes ficheiros de áudio são representados no espaço vetorial por cada modelo, utilizámos uma técnica de redução de dimensionalidade chamada **t-SNE (t-Distributed Stochastic Neighbor Embedding)**. Esta técnica permite transformar os embeddings de alta dimensão (por exemplo, 768 ou 1024 dimensões) em dois eixos principais, possibilitando a visualização num gráfico 2D.

---

#### 🧠 O que significa a visualização?

- **Pontos próximos** no gráfico indicam que os ficheiros de áudio correspondentes geraram embeddings semelhantes.
  - Isto pode significar que o modelo está a conseguir **captar padrões semelhantes** entre áudios que partilham características comuns (por exemplo, o mesmo género musical, tipo de voz ou ambiente).
  
- **Pontos afastados** indicam que os ficheiros foram considerados diferentes pelo modelo, ou seja, os seus embeddings têm **baixa similaridade**.
  - Isso é esperado entre ficheiros com conteúdo muito distinto (ex: música vs. fala).

---

#### 🔍 Como foi feito?

```python
modelo_tsne = 'wav2vec2'
subset = df[df['modelo'] == modelo_tsne]


In [10]:
# 🧠 Visualização t-SNE com validação
modelo_tsne = 'wav2vec2'
subset = df[df['modelo'] == modelo_tsne]
if len(subset) >= 2:
    try:
        emb = np.stack(subset['embedding'].values)
        tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(emb)-1))
        emb_2d = tsne.fit_transform(emb)
        fig = px.scatter(x=emb_2d[:,0], y=emb_2d[:,1], text=subset['ficheiro'], title=f't-SNE ({modelo_tsne})')
        fig.show()
    except Exception as e:
        print(f'Erro no t-SNE: {e}')
else:
    print('Dados insuficientes para gerar t-SNE.')

### 9️⃣ Pesquisa Vetorial Ad-hoc com Milvus

Neste Jupyter Notebook é possível realizar **pesquisas por similaridade** de áudio utilizando embeddings gerados pelos modelos analisados. Esta funcionalidade permite procurar ficheiros semelhantes ao áudio fornecido — seja um ficheiro novo carregado pelo utilizador ou um ficheiro já presente na base de dados.

---

#### ⚙️ Pré-requisitos

Para que a pesquisa funcione corretamente, é necessário que o **servidor Milvus esteja ativo**. Caso contrário, o sistema não conseguirá aceder às coleções de vetores armazenadas.

Para levantar o Milvus localmente, recomenda-se o seguinte comando Docker:

```bash
docker-compose up -d

Como funciona a pesquisa
Escolha do modelo: O utilizador seleciona qual modelo de embeddings deseja usar para gerar a representação vetorial do áudio.

Entrada de áudio:

Pode ser feito o upload de um ficheiro .wav, .mp3, .flac, etc.

Ou selecionado um ficheiro já existente no dataset.

Extração do embedding: Dependendo do modelo, o áudio é processado para gerar o vetor de características.

Pesquisa por similaridade: Esse vetor é comparado com todos os vetores presentes na coleção correspondente no Milvus, retornando os ficheiros mais semelhantes com base na distância vetorial.

Visualização dos resultados: Os resultados da pesquisa são apresentados numa tabela e num gráfico de barras com as distâncias de similaridade.

📌 Observações importantes
Por limitações do Jupyter Notebook, os modelos yamnet e vggish podem não funcionar corretamente aqui devido à forma como carregam os módulos do TensorFlow Hub.

No entanto, estes modelos funcionam normalmente na aplicação desenvolvida com Streamlit, que oferece maior compatibilidade e desempenho na extração e inferência.

✅ Funcionalidades da Interface
Upload de ficheiro novo ou seleção de ficheiro existente;

Escolha do modelo com lista suspensa interativa;

Seleção da coleção do Milvus ou uso automático da coleção padrão por modelo;

Visualização dos resultados com distâncias em gráfico interativo.

📚 Benefícios desta abordagem
Permite testar a eficácia real dos embeddings, na prática, para pesquisa vetorial;

Ajuda a entender a capacidade de generalização de cada modelo;

Facilita a construção de sistemas de recuperação de informação por áudio com base em embeddings semânticos.



In [11]:
# 📤 Pesquisa Ad-hoc (Upload ou Ficheiro da Base de Dados) e funções auxiliares
import tempfile
import librosa
import numpy as np
import torch
import pandas as pd
import plotly.express as px
import ast
import openl3
import io
import os

# Import transformers models
from transformers import (
    Wav2Vec2Processor, Wav2Vec2Model,
    ClapProcessor, ClapModel,
    ASTFeatureExtractor, ASTModel
)

import tensorflow as tf
import tensorflow_hub as hub

# Função para extrair embedding de um áudio
def extract_embedding(model_name, temp_path):
    audio, sr = librosa.load(temp_path, sr=None)
    
    if model_name == "wav2vec2":
        processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
        model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        # Resample if needed
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()[0]
    
    elif model_name == "clap":
        processor = ClapProcessor.from_pretrained("laion/clap-htsat-unfused")
        model = ClapModel.from_pretrained("laion/clap-htsat-unfused")
        # Resample if needed
        if sr != 48000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=48000)
        inputs = processor(audios=audio, sampling_rate=48000, return_tensors="pt")
        with torch.no_grad():
            embeddings = model.get_audio_features(**inputs)
        return embeddings[0].numpy()
    
    elif model_name == "ast":
        fe = ASTFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
        model = ASTModel.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
        # Resample if needed
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        inputs = fe(audio, sampling_rate=16000, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()[0]
    
    elif model_name == "vggish":
        # Mesmo código usado no Streamlit que funciona
        import tensorflow_hub as hub
        # Resample para 16kHz conforme exigido pelo VGGish
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        # Converter para mono se for estéreo
        if len(audio.shape) > 1:
            audio = np.mean(audio, axis=1)
        # Carregar modelo e obter embedding
        vggish_model = hub.load('https://tfhub.dev/google/vggish/1')
        embedding = vggish_model(audio).numpy()
        # Fazer média se tivermos múltiplos embeddings
        if embedding.ndim > 1:
            embedding = np.mean(embedding, axis=0)
        return embedding
    
    elif model_name == "yamnet":
        # Mesmo código usado no Streamlit que funciona
        import tensorflow_hub as hub
        import tensorflow as tf
        # Resample para 16kHz conforme exigido pelo YAMNet
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        # Converter para tensor do TensorFlow
        waveform = tf.convert_to_tensor(audio, dtype=tf.float32)
        # Carregar modelo e obter embedding
        yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')
        scores, embeddings, spectrogram = yamnet_model(waveform)
        # Fazer média dos embeddings se tivermos múltiplos
        embedding = np.mean(embeddings.numpy(), axis=0)
        return embedding
    
    elif model_name == "openl3":
        audio = np.mean(audio, axis=1) if audio.ndim > 1 else audio
        emb, _ = openl3.get_audio_embedding(audio, sr, content_type="music", embedding_size=512)
        return emb.mean(axis=0) if emb.ndim > 1 else emb.squeeze()
    
    else:
        raise ValueError("Modelo não suportado")

# Função para pesquisa real a partir do banco de dados
def real_search_from_db(df, client, modelo_milvus, collection_name, k):
    dimensoes_modelos = {
        "ast": 768, "clap": 512, "yamnet": 1024,
        "openl3": 512, "vggish": 128, "wav2vec": 768, "wav2vec2": 768,
    }
    dim_esperada = dimensoes_modelos.get(modelo_milvus)
    if not dim_esperada:
        raise ValueError(f"Dimensão desconhecida para modelo '{modelo_milvus}'")

    ficheiros = df[(df['modelo'] == modelo_milvus) & (df['embedding'].apply(lambda x: len(x) == dim_esperada))]['ficheiro'].tolist()
    if not ficheiros:
        raise ValueError(f"Sem ficheiros com dimensão {dim_esperada} para modelo '{modelo_milvus}'")

    row_query = df[(df['modelo'] == modelo_milvus) & (df['ficheiro'] == ficheiros[0])].iloc[0]
    embedding_query = row_query['embedding']

    result = client.search(
        collection_name=collection_name,
        data=[embedding_query],
        limit=k,
        output_fields=["id", "vector"]
    )
    hits = result[0] if result else []
    res_df = pd.DataFrame([{"id": h.get("id"), "distance": h.get("distance")} for h in hits])
    return res_df

# Interface de usuário
upload_widget = widgets.FileUpload(accept='.wav,.mp3,.flac,.ogg', multiple=False)
modelo_options = ["wav2vec2", "clap", "ast", "vggish", "yamnet", "openl3"]
modelo_widget = widgets.Dropdown(options=modelo_options, description="Modelo")
usar_ficheiro_existente = widgets.Checkbox(value=False, description="Usar ficheiro do dataset")
ficheiro_select = widgets.Dropdown(description="Ficheiro", options=[])

# Widget para seleção de coleção
collection_widget = widgets.Dropdown(description="Coleção", options=[])
usar_colecao_padrao = widgets.Checkbox(value=True, description="Usar coleção padrão")

output = widgets.Output()

# Função para atualizar opções de coleção
def update_collection_options(*args):
    try:
        collections = client.list_collections() if client else []
        if collections:
            collection_widget.options = collections
    except Exception as e:
        with output:
            print(f"Erro ao atualizar coleções: {e}")

# Função para atualizar opções de arquivo
def update_file_options(*args):
    if usar_ficheiro_existente.value:
        ficheiro_select.options = df[df['modelo'] == modelo_widget.value]['ficheiro'].tolist()
        ficheiro_select.layout.display = 'block'
    else:
        ficheiro_select.layout.display = 'none'
        # Exibe aviso para VGGish e YAMNet quando esses modelos são selecionados
        modelo_atual = modelo_widget.value

# Função para atualizar visibilidade do seletor de coleção
def update_collection_visibility(*args):
    if usar_colecao_padrao.value:
        collection_widget.layout.display = 'none'
    else:
        collection_widget.layout.display = 'block'
        # Atualize as opções quando o widget ficar visível
        update_collection_options()

def perform_search(b):
    with output:
        output.clear_output()
        try:
            query_id = None  # Initialize a variable to track query ID
            
            if usar_ficheiro_existente.value:
                if not ficheiro_select.value:
                    print("Por favor, selecione um ficheiro do dataset.")
                    return
                    
                # Usar ficheiro do dataset
                row = df[(df['modelo'] == modelo_widget.value) & (df['ficheiro'] == ficheiro_select.value)].iloc[0]
                embedding = np.array(row['embedding'])
                query_id = row.get('id')  # Get ID if available
                print(f"Usando embedding do ficheiro: {ficheiro_select.value}")
            
            else:
                # Este bloco estava dentro do if anterior, corrigindo a indentação
                if not upload_widget.value:
                    print("Por favor, carregue um ficheiro de áudio.")
                    return
                
                # Usar ficheiro carregado
                file_content = upload_widget.value[0]['content']
                file_bytes = io.BytesIO(file_content)
                file_name = upload_widget.value[0]['name']
                
                with tempfile.NamedTemporaryFile(delete=False, suffix=f"_{file_name}") as f:
                    f.write(file_bytes.read())
                    temp_path = f.name
                
                print(f"Extraindo embedding com modelo {modelo_widget.value}...")
                embedding = extract_embedding(modelo_widget.value, temp_path)
                os.unlink(temp_path)  # Remove o arquivo temporário
            
            # Determinar qual coleção usar
            if usar_colecao_padrao.value:
                collection_name = f"audio_{modelo_widget.value}"
            else:
                collection_name = collection_widget.value
                if not collection_name:
                    print("Por favor, selecione uma coleção.")
                    return
            
            # Verificar se a coleção existe
            if collection_name not in client.list_collections():
                print(f"Coleção {collection_name} não existe no Milvus!")
                return
                
            print(f"Pesquisando na coleção {collection_name}...")
            if query_id is not None:
                expr = f"id != {query_id}"
                result = client.search(
                    collection_name=collection_name,
                    data=[embedding.tolist()],
                    limit=6,  # Request one more result than needed
                    filter=expr,
                    output_fields=['id', 'vector']
                )
            else:
                result = client.search(
                    collection_name=collection_name,
                    data=[embedding.tolist()],
                    limit=6,  # Request one more to account for possible self-match
                    output_fields=['id', 'vector']
                )
            
            hits = result[0] if result else []
            if not hits:
                print("Nenhum resultado encontrado.")
                return
                
            df_res = pd.DataFrame([{'id': h.get('id'), 'distance': h.get('distance')} for h in hits])
            display(df_res)
            
            # Visualização dos resultados
            fig = px.bar(df_res, x='id', y='distance', title=f'Resultados da Pesquisa - Coleção: {collection_name}')
            fig.show()
            
        except NotImplementedError as e:
            print(f"⚠️ {str(e)}")
        except Exception as e:
            import traceback
            print(f"Erro ao processar ou pesquisar: {str(e)}")
            traceback.print_exc()

# Configuração da interface
search_button = widgets.Button(description="Pesquisar")
search_button.on_click(perform_search)
usar_ficheiro_existente.observe(update_file_options, names='value')
modelo_widget.observe(update_file_options, names='value')
usar_colecao_padrao.observe(update_collection_visibility, names='value')

# Inicializar visibilidade
ficheiro_select.layout.display = 'none'
collection_widget.layout.display = 'none'  # Esconde o seletor de coleção inicialmente

# Atualizar coleções disponíveis no início
update_collection_options()

# Exibir widgets
display(widgets.VBox([
    widgets.HBox([modelo_widget, usar_ficheiro_existente]),
    widgets.HBox([usar_colecao_padrao, collection_widget]),
    upload_widget,
    ficheiro_select,
    search_button,
    output
]))

# Inicializar opções de ficheiros
update_file_options()